In [314]:
import random
import networkx as nx
import itertools
import numpy as np
import pandas as pd

In [315]:
def mc_upper_bound(G):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "chromatic_number" integer upper bound on the maximum clique number
	"""
	answ = nx.algorithms.coloring.greedy_color(G)
	chromatic_number = list(set(list(answ.values())))
	return len(chromatic_number)

In [316]:
min_grade = 467/613
max_grade = 607/613
print(f"Valore più basso di grafo in rapporto al totale grafo originale: {min_grade}")
print(f"Valore più alto di grafo in rapporto al totale grafo originale: {max_grade}")

Valore più basso di grafo in rapporto al totale grafo originale: 0.7618270799347472
Valore più alto di grafo in rapporto al totale grafo originale: 0.9902120717781403


# Funzione aggiornata
### In questa seconda versione vengono scelti dei valori massimi e minimi di rapporto tra grado di un nodo e numero di nodi totali. Cerca di creare archi per un nodo in maniera che il suo grado sia un valore random nel range scelto o finchè può collegarlo senza aumentare l'upper bound

In [317]:
# Funzione ricorsiva che sceglie un nodo random che non sia v o un nodo nella lista di nodi non utilizzabili
#Ritorna "None" se sono già stati controllati tutti i nodi così da bloccare il ciclo
def random_node(nodes, v, no_node_list):
    node = random.choice(list(nodes))
    if len(no_node_list) >= len(nodes)-1:
        return None
    elif node != v and node not in no_node_list:
        return node
    else:
        return random_node(nodes, v, no_node_list)

In [318]:
# Crea un grafo con "num_nodes" = numero nodi, con "dim_cli" nodi di cricca e "num_cli" cricche, poi collega i nodi delle cricche
def create_graph_with_rand_clique(num_nodes, dim_cli, num_cli, random_clique_dimension):
    g = nx.Graph()
    cliques_nodes = []
    for i in range(num_nodes):
        g.add_node(i)
    nodes = list(g.nodes())
    for k in range(num_cli):
        if random_clique_dimension:
            clique = random.sample(nodes, random.randint(2,dim_cli))
        else:
            clique = random.sample(nodes, dim_cli)
        cliques_nodes.append(sorted(clique))
        nodes = [a for a in nodes if a not in clique]
        combinations = itertools.combinations(clique,2)
        g.add_edges_from(combinations)
    return g, cliques_nodes, nodes

In [319]:
# "num_node" = numero nodi cricca, 
# "ex_node" nodi non appartenenti alla cricca, 
# "grade_max_min" grado massimo e minimo tra 0 e 1
# "add_edges_between_cliques" il nome spiega, meglio mettere a True
# "random_clique_dimension" setta la dim della cricca random con massimo "num_node", per ora settare a False
def graph_with_clique(num_node, ex_node, num_clique, grade_min, grade_max, add_edges_between_cliques, random_clique_dimension):
    g, clique_nodes, list_ex_nodes = create_graph_with_rand_clique(num_node*num_clique+ex_node, num_node, num_clique, random_clique_dimension)
    total_nodes = len(g.nodes())
    ub = mc_upper_bound(g)
    for i in list_ex_nodes:
        node_grade = random.uniform(grade_min, grade_max) #numero di iterazioni
        no_nodes_1 = []
        while (g.degree()[i]/total_nodes) < node_grade: #finchè sto sotto il grado del nodo continuo
            node = random_node(g.nodes, i, no_nodes_1) #scelgo nodo
            if node is None: #Se ho usato tutti i nodi fermo 
                break
            g.add_edge(i, node) #aggiunge arco
            new_ub = mc_upper_bound(g) #ricalcola upper bound
            if new_ub > ub: #rimuove arco se necessario
                g.remove_edge(i, node)
            no_nodes_1.append(node) #aggiungo nodo a quelli già usati
    if add_edges_between_cliques:
        for cli in clique_nodes:
            ok_nodes = [a for a in g.nodes() if a not in cli]
            for n in cli:
                node_grade = random.uniform(grade_min, grade_max) #numero di iterazioni
                no_nodes_2 = []
                while (g.degree()[n]/total_nodes)< node_grade: #finchè sto sotto il grado del nodo continuo
                    node = random_node(ok_nodes, n, no_nodes_2) #scelgo nodo
                    if node is None: #Se ho usato tutti i nodi fermo
                        break
                    g.add_edge(n, node) #aggiunge arco
                    new_ub = mc_upper_bound(g) #ricalcola upperbound
                    if new_ub > ub: #rimuove arco se necessario
                        g.remove_edge(n, node)
                    no_nodes_2.append(node) #aggiungo nodo a quelli già usati
    return g, clique_nodes

# Test

In [320]:
G, cli = graph_with_clique(20,100,1, min_grade, max_grade, True, False)

In [321]:
degrees = dict(G.degree())
print(f"Rapporto grado minimo:{min(degrees.values())/len(G)}")
print(f"Rapporto grado massimo:{max(degrees.values())/len(G)}")

Rapporto grado minimo:0.725
Rapporto grado massimo:0.975


# Funzione di assegnazione pesi
### Input grafo e i due dataframe di probabilità
### Output un dict che contiene vertice:[w00,w01]
### Viene sviluppata così in maniera da poter estrarre più set di pesi per lo stesso grafo, il dict verrà passato poi alla funzione di assegnazione pesi

In [373]:
def assegna_pesi_casuali(grafo, df0, df1):
    # Calcola il numero totale di nodi nel grafo
    num_nodi = grafo.number_of_nodes()
    df_weights = {}
    
    # Itera su ogni nodo del grafo
    for nodo in grafo.nodes():
        # Calcola il rapporto grado/numero di nodi per il nodo attuale
        grado = grafo.degree[nodo]
        rapporto = round(grado / num_nodi, 3)
        
        # Trova la riga nel dataframe w00 corrispondente al rapporto più vicino
        if rapporto in df0.index:
            riga0 = df0.loc[rapporto]
        else:
            idx_rapporto_piu_vicino0 = pd.DataFrame(df0.index - rapporto).abs().idxmin()
            rapporto_piu_vicino0 = df0.index[idx_rapporto_piu_vicino0]
            riga0 = df0.loc[rapporto_piu_vicino0[0]]
        # Trova la riga nel dataframe w01 corrispondente al rapporto più vicino
        if rapporto in df1.index:
            riga1 = df1.loc[rapporto]
        else:
            idx_rapporto_piu_vicino1 = pd.DataFrame(df0.index - rapporto).abs().idxmin()
            rapporto_piu_vicino1 = df0.index[idx_rapporto_piu_vicino1]
            riga1 = df1.loc[rapporto_piu_vicino1[0]]
        
        # Estrai la probabilità dalla riga e normalizzala se necessario
        probabilita0 = riga0.values
        pesi0 = riga0.index
        probabilita1 = riga1.values
        pesi1 = riga1.index
        
        # Scegli un peso casualmente secondo la distribuzione di probabilità
        peso_scelto0 = np.random.choice(pesi0, p=probabilita0)
        peso_scelto1 = np.random.choice(pesi1, p=probabilita1)
        
        #aggiungo al dataframe il nodo e peso
        df_weights[nodo] = [peso_scelto0, peso_scelto1]
    
    return df_weights

## Test

In [374]:
matrice_w00 = pd.read_csv('./prob_w00.csv', index_col=0)
matrice_w01 = pd.read_csv('./prob_w00.csv', index_col=0)

In [375]:
G, cli = graph_with_clique(40, 70, 1, min_grade, max_grade, True, False)

In [376]:
assegna_pesi_casuali(G, matrice_w00, matrice_w01)

{0: ['0.0', '0.0'],
 1: ['0.2674', '0.2661'],
 2: ['0.0032', '0.009'],
 3: ['0.0209', '0.0208'],
 4: ['0.0208', '0.0196'],
 5: ['0.0', '0.0'],
 6: ['1.0', '1.0'],
 7: ['0.0251', '0.0233'],
 8: ['0.0196', '0.0209'],
 9: ['0.2697', '0.2674'],
 10: ['0.0312', '0.0045'],
 11: ['0.0056', '0.0056'],
 12: ['0.0096', '0.0099'],
 13: ['0.0153', '0.0153'],
 14: ['0.0062', '0.004'],
 15: ['0.2674', '0.2674'],
 16: ['0.0308', '0.0308'],
 17: ['0.0099', '0.0099'],
 18: ['0.0', '0.0'],
 19: ['0.0419', '0.0411'],
 20: ['0.0153', '0.0153'],
 21: ['0.0419', '0.0419'],
 22: ['0.9968', '0.9976'],
 23: ['0.2688', '0.2674'],
 24: ['0.2693', '0.267'],
 25: ['0.0899', '0.0238'],
 26: ['0.0153', '0.0153'],
 27: ['0.2693', '0.2666'],
 28: ['1.0', '0.9976'],
 29: ['0.9945', '0.9968'],
 30: ['0.0027', '0.0312'],
 31: ['0.0114', '0.0114'],
 32: ['0.0056', '0.0056'],
 33: ['0.9945', '0.9945'],
 34: ['0.0014', '0.0014'],
 35: ['1.0', '1.0'],
 36: ['0.1847', '0.1847'],
 37: ['0.0646', '0.0045'],
 38: ['0.0114', '0.0